In [36]:
import pandas as pd
import numpy as np
import os
import datasets
np.random.seed(270)

In [9]:
def read_dfs(data_name, suffixes=["train", "val", "test"]):
    print(f"loading data: {data_name} ...")
    data = pd.DataFrame()
    for i in range(len(suffixes)):
        suffix = suffixes[i]
        data_sfx = pd.read_json(f"{data_name}_{suffix}.jsonl", lines=True)
        data = pd.concat([data, data_sfx], axis=0)
    return data

In [10]:
epfl_dpo_data = read_dfs("EPFL/raw/EPFL_DPO", suffixes=["train_clipped", "test", "val"])
epfl_sft_data = read_dfs("EPFL/raw/EPFL_SFT", suffixes=["train_clipped", "test", "val"])
helpsteer_data = read_dfs("helpSteer/raw/helpsteer", suffixes=["train_clipped", "test", "val"])
mathqa_data = read_dfs("mathQA/raw/mathQA", suffixes=["train_clipped", "val", "test"])
mmlu_data = read_dfs("MMLU/raw/MMLU", suffixes=["train_clipped", "test_clipped", "val_clipped"])
openqa_data = read_dfs("openQA/raw/openQA", suffixes=["train_clipped", "val", "test"])
shp_data = read_dfs("shp/raw/shp", suffixes=["train_clipped", "test_clipped", "val_clipped"])

loading data: EPFL/raw/EPFL_DPO ...
loading data: EPFL/raw/EPFL_SFT ...
loading data: helpSteer/raw/helpsteer ...
loading data: mathQA/raw/mathQA ...
loading data: MMLU/raw/MMLU ...
loading data: openQA/raw/openQA ...
loading data: shp/raw/shp ...


In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2",
                                              trust_remote_code=True,
                                              padding_side='left',
                                              add_eos_token=True,
                                              add_bos_token=True,
                                              use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

/home/grapix/miniconda3/envs/m3-mnlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [37]:
def formatting_func(example, field):
    text = f"{example[field]}"
    return text

def generate_and_tokenize_prompt(prompt, field):
    return tokenizer(formatting_func(prompt, field))

def load_dataset(data_path):
    dataset = pd.read_json(data_path, lines=True)
    dataset = dataset.Dataset.from_pandas(dataset)

    return dataset

def tokenize_dataset(dataset, field):
    tokenized_dataset = dataset.map(lambda x: generate_and_tokenize_prompt(x, field))

    return tokenized_dataset

def compute_lengths(dataframe, field):
    # tokenization and lengths computation
    dataset = datasets.Dataset.from_pandas(dataframe)
    tokenized_dataset = tokenize_dataset(dataset, field)
    lengths = [len(x[field]) for x in tokenized_dataset]

    return lengths

In [41]:
def process_dataframe(dataframe, data_path, fname, field="prompt", max_length=512):
    # if data_path directory does not exist, create it
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    
    print(f"processing dataframe {fname}...")

    # shuffling the dataframe
    dataframe = dataframe.sample(frac=1).reset_index(drop=True)
    print(f"Length before clipping: {len(dataframe)}")

    # lengths computation 
    lengths = compute_lengths(dataframe, field)

    # filtering out the examples that exceed the max_length
    dataframe["lengths"] = lengths
    dataframe = dataframe[dataframe["lengths"] <= max_length]
    dataframe = dataframe.drop(columns=["lengths"])
    print(f"Length after clipping: {len(dataframe)}\n")

    #splitting 60-train / 20-test / 20-val
    train_size = int(0.6 * len(dataframe))
    val_size = int(0.2 * len(dataframe))

    train_df = dataframe[:train_size]
    val_df = dataframe[train_size:train_size+val_size]
    test_df = dataframe[train_size+val_size:]

    # saving the dataframes
    train_df.to_json(f"{data_path}_{fname}_train.jsonl", orient="records", lines=True)
    val_df.to_json(f"{data_path}_{fname}_val.jsonl", orient="records", lines=True)
    test_df.to_json(f"{data_path}_{fname}_test.jsonl", orient="records", lines=True)

    return 0

In [42]:
# shuffling the data
np.random.seed(270)

process_dataframe(epfl_dpo_data, "EPFL/processed/", "EPFL_DPO", field="prompt")
process_dataframe(epfl_sft_data, "EPFL/processed/", "EPFL_SFT", field="prompt")
process_dataframe(helpsteer_data, "helpSteer/processed/", "helpsteer", field="prompt")
process_dataframe(mathqa_data, "mathQA/processed/", "mathQA", field="question")
process_dataframe(mmlu_data, "MMLU/processed/", "MMLU", field="question")
process_dataframe(openqa_data, "openQA/processed/", "openQA", field="question")
process_dataframe(shp_data, "shp/processed/", "shp", field="prompt")

processing dataframe EPFL_DPO...
Length before clipping: 12614


Map: 100%|██████████| 12614/12614 [00:21<00:00, 577.93 examples/s]


Length after clipping: 9188
processing dataframe EPFL_SFT...
Length before clipping: 8378


Map: 100%|██████████| 8378/8378 [00:15<00:00, 535.87 examples/s]


Length after clipping: 6149
processing dataframe helpsteer...
Length before clipping: 15860


...

Length after clipping: 10680
processing dataframe mathQA...
Length before clipping: 33426


Map: 100%|██████████| 33426/33426 [00:42<00:00, 789.45 examples/s] 


Length after clipping: 33121
processing dataframe MMLU...
Length before clipping: 105488


Map:   2%|▏         | 1926/105488 [00:09<08:49, 195.61 examples/s]


KeyboardInterrupt: 

In [19]:
# split the data 60 40
train_data = mmlu_data.iloc[:int(0.6*len(mmlu_data))]
val_data = mmlu_data.iloc[int(0.6*len(mmlu_data)):int(0.8*len(mmlu_data))]
test_data = mmlu_data.iloc[int(0.8*len(mmlu_data)):]

train_data.to_json("MMLU/processed/MMLU_train.jsonl", orient="records", lines=True)
val_data.to_json("MMLU/processed/MMLU_val.jsonl", orient="records", lines=True)
test_data.to_json("MMLU/processed/MMLU_test.jsonl", orient="records", lines=True)

,id,subject,question,answer,answer_text,choices
0,101112,race,"Billy Dengler, a 14-year-old boy, is in the ei...",A,a screen reader,"[a screen reader, his mother, a computer, his ..."
1,91724,race,"One day, a little monkey and a little fox went...",B,eat the apples,"[build a bridge, eat the apples, pull down a t..."
2,103336,race,"More than 50,000,000 people live in the rainfo...",B,the rainforest people have done something to p...,"[we need wood to build houses, so we have to c..."
3,31870,race,"Since I became a full--time freelancer , I've ...",D,He has difficulty getting up early.,"[He has a full--time job outside., He likes ly..."
4,71094,race,"The researchers, led by Hwang Woo-suk, insist ...",B,dog,"[cat, dog, cow, goat]"
...,...,...,...,...,...,...
115410,35285,race,Many of today's parents fondly remember childh...,D,get the chance to explore and create ideas,[learn to share and cooperate with one another...
115411,59933,race,Long live language. That's the message Edmonto...,C,protecting languages and cultures from extinct...,"[mastering more than one foreign language., pu..."
115412,56655,race,"Are there fastest ways to lose weight? Yes, th...",C,red and orange,"[black and yellow, gray and white, red and ora..."
115413,57609,race,South Asia heatwave kills nearly 100 DHAKA - A...,B,Talcher,"[Dhaka, Talcher, Andhra Pradesh, Nepal]"
